<a href="https://colab.research.google.com/github/manojreddxy/python_nlp/blob/main/csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [11]:
import pandas as pd # Make sure this line is at the top of your notebook or cell.

df = pd.read_csv('/content/covid_fake.csv')

In [12]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [16]:
df.shape

(1164, 4)

In [15]:

print(df.columns)

Index(['title', 'text', 'source', 'label'], dtype='object')


In [14]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [18]:
df['label'] = df['label'].str.replace('fake', 'Fake', case=False)  # Standardize 'label' column
df['source'] = df['source'].str.replace('facebook', 'Facebook', case=False)  # Standardize 'source' column
df['text'] = df['text'].fillna(df['title'])  # Fill missing 'text' with 'title'
df.loc[5, 'label'] = 'Fake'  # Set specific label values using .loc with row index and column name
df.loc[15, 'label'] = 'True'
df.loc[43, 'label'] = 'Fake'
df.loc[131, 'label'] = 'True'
df.loc[242, 'label'] = 'Fake'
df['title'] = df['title'].fillna('missing')  # Fill missing 'title' with 'missing'
df['source'] = df['source'].fillna('missing')  # Fill missing 'source' with 'missing'

In [21]:
df.isna()

,title,text,source,label
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
1159,False,False,False,False
1160,False,False,False,False
1161,False,False,False,False
1162,False,False,False,False


In [20]:
df.head()


,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,missing,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,missing,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,missing,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,missing,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [25]:
# Create a new column 'title_text' by combining 'title' and 'text'
df['title_text'] = df['title'] + ' ' + df['text']

# Now you can access df['title_text'][1]
print(df['title_text'][1])

missing Hydroxychloroquine has been shown to have a 100% effective rate treating COVID-19. Yet Democrat Gretchen Whitmer is threatening doctors who prescribe it. If Trump is for something, Democrats are against it. They are okay with people dying if it means opposing Trump.


In [28]:
import re # Import the 're' module for regular expressions

def preprocessor(text):
  text=re.sub('<[^>]*>','',text)
  text=re.sub(r'[^\w\s]>','',text)
  text=re.sub(r'[\n]','',text)
  text=text.lower()
  return text
# Apply the preprocessor function to the 'title_text' column (note the corrected column name)
df['title_text'] = df['title_text'].apply(preprocessor)
df['title_text'][3] # Access the 'title_text' column (note the corrected column name)

'missing the corona virus is a man made virus created in a wuhan laboratory. ask @billgates who financed it.'

In [31]:
!pip install nltk
import nltk # This line imports the nltk library
nltk.download('punkt') # This line downloads necessary resources
from nltk.stem.porter import PorterStemmer # This line imports the PorterStemmer class

porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer # Import TfidfVectorizer from the correct module

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                        tokenizer=tokenizer_porter,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True)
x=tfidf.fit_transform(df['title_text']) # Changed 'tittle_text' to 'title_text'
y=df.label.values

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [40]:

X=tfidf.fit_transform(df['title_text'])
X.shape

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(1164, 47301)

In [42]:
from sklearn.model_selection import train_test_split # Import the train_test_split function

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0, \
                                               test_size=0.3,shuffle=False)

In [44]:
from sklearn.linear_model import LogisticRegressionCV # Import LogisticRegressionCV

clf=LogisticRegressionCV(cv=5,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=300)
clf.fit(X_train,y_train)
fake_news_model=open('fake_news_model.pkl','wb')
import pickle # Importing pickle here for better code organization
pickle.dump(clf,fake_news_model)
fake_news_model.close()

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


In [47]:
from sklearn.metrics import classification_report, accuracy_score # Corrected module name to 'metrics'
y_pred = clf.predict(X_test)
print("---Test Set Results---")
print(classification_report(y_test, y_pred)) # Corrected function name to 'classification_report'

---Test Set Results---
              precision    recall  f1-score   support

        Fake       0.92      0.92      0.92       132
        TRUE       0.95      0.94      0.95       218
        True       0.00      0.00      0.00         0

    accuracy                           0.94       350
   macro avg       0.62      0.62      0.62       350
weighted avg       0.94      0.94      0.94       350



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
